**420-A58-SF - Algorithmes d'apprentissage non supervisé - Hiver 2023 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2023 Mikaël Swawola
<br/>
![Travaux Pratiques - Regroupement hiérachique](static/01-03-banner.png)
<br/>
**Objectif:** cette séance de travaux pratiques a pour objectif l'analyse de données génomiques (jeu de données NCI60). Les techniques mise en oeuvre seront l'ACP et le regroupement hiérarchique

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

# Le reste des modules sera importé au fur et à mesure des exercices ...

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
# Configuration de la visualisation
sns.set(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams['figure.figsize']=(12,8)

## 1 - Lecture des données

Les méthodes d'apprentissage non supervisé comme l'ACP et le clustering sont très populaires en analyse des données génomiques. Ces méthodes seront illustrées sur le jeu de données de biopuces NCI60, qui
consiste en 6830 **mesures d'expression de gènes** sur 64 lignées de cellules cancéreuses.

**Exercice 1-1 - À l'aide de la librairie Pandas, lire le fichier de données `NCI60.csv`**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code

**Exercice 1-2 - Combien d'observations et de variables compte le jeu de données ? Que constatez vous ?**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code

**Exercice 1-3 - Une chance, ce jeu de données est fourni avec le type de cancer pour chaque cellule cancéreuse. Lire le fichier `NCI60-labels.csv`**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code

**Exercice 1-4 - Selon vous, est-il nécéssaire de mettre à l'échelle (standardiser ou normaliser) le jeu de données ? Justifier votre réponse.**

In [ ]:
# Votre réponse ici

## 2 - Analyse en composantes principales (ACP)

**Exercice 2-1 - À l'aide de la libraire Scikit-learn, effectuer une analyse en composantes principales. Vérifier la proportion de variance (PVE) et afficher sur un graphique les deux premières composantes et le type de cancer. Que constatez-vous ?**

In [ ]:
# Compléter cette cellule (ACP) ~ 3 lignes de code

In [ ]:
# Compléter cette cellule (variance) ~ 1 ligne de code

In [ ]:
# Compléter cette cellule (affichage) ~ 1 ligne de code

## 3 - Regroupement hiérarchique

Nous effectuons maintenant le regroupement hiérarchique à l'aide des 4 méthodes de lien vus en cours: Ward, Complete, Average et Single. La distance euclidienne est utilisée comme mesure de dissimilarité. Pour chaque lien, afficher le dendrogramme correspondant (voir `helpers.py`)

**Exercice 3-1 - Linkage par défaut (Ward)**

In [ ]:
# Compléter cette cellule ~ 2 lignes de code

**Exercice 3-2 - Linkage Complete**

In [ ]:
# Compléter cette cellule ~ 2 lignes de code

**Exercice 3-3 - Linkage average**

In [ ]:
# Compléter cette cellule ~ 2 lignes de code

**Exercice 3-4 - Linkage Single**

In [ ]:
# Compléter cette cellule ~ 2 lignes de code

**Exercice 3-5 - Compte tenu des résultats précédents, choisir une méthode de lien et seuiller le dendrogramme à la hauteur "optimale"**

In [ ]:
# Compléter la cellule ~ 2 lignes de code

## 4 - Partitionnement K-moyennes

**Exercice 4 - Comparer les résultats précédents avec le partitionnement K-moyennes**

In [ ]:
# Compléter la cellule ~ 5-10 lignes de code

## 5 - Débruitage des données par ACP (optionnel)

**Exercice 5 - Effectuer le regroupement hiérarchique directement sur les premières composantes principales. Comparer les résultats obtenus avec ceux du jeu de données "complet"**

In [ ]:
# Compléter la cellule ~ X lignes de code

## 6 - Références

Cet atelier est grandement inspiré du Lab 3: NCI60 Data Example (Introduction to statistical learning)

**Fin de l'atelier 01-03**